# About this kernel

This offers a boilerplate for a complete workflow for the Severstal Steel Defect Detection challenge. We will be using the popular U-Net architecture* in Keras, but I made this to be as extensible as possible so that you can experiment with different types of models. It is broken down in the following sections:

* **Preprocessing**: Expand the train dataframe to include image ID. Also create `mask_count_df` which will be useful for later.
* **Utility Functions**: Mostly copied from Paul's kernel and SIIM starter code (see references). You won't need to modify those.
* **Sample Test**: Simply visualizing a sample image and its masks,
* **Data Generator**: Very long and possibly complex. If you can, skip this part of the code. **If you absolute need to modify the data generation process, please take a look `__generate_X` and `__generate_y`**; in theory everything else should be left as is. 
* **Model Architecture**: The architecture is slightly different from the other kernels, since **it learns to predict all of the four masks at the same time**, instead of predicting a single mask and duplicating it. It also takes as input grayscale images.
* **Training**: Running only for 9 epochs due to the time constraints (60 mins, this is roughly 300s per epoch).
* **Evaluation & Submission**: The submission code is pretty messy. Essentially, I'm splitting the test dataframe into multiple chunks, then run the model and `mask2rle` converter on the results. I'm doing this in order to not run out of RAM as we try to convert all the masks from array to RLE.


## Changelog

V15: Added BCE-Dice loss, which should give slightly more accurate loss than pure binary cross-entropy (BCE).

## References

* Data generator: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
* RLE encoding and decoding: https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
* Architecture: https://www.kaggle.com/jesperdramsch/intro-chest-xray-dicom-viz-u-nets-full-data
* Mask encoding: https://www.kaggle.com/c/siim-acr-pneumothorax-segmentation/data
* Source for `bce_dice_loss`: https://lars76.github.io/neural-networks/object-detection/losses-for-segmentation/

In [ ]:
import os
import json

import cv2
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.losses import binary_crossentropy
from keras.callbacks import Callback, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers

# Preprocessing

In [ ]:
train_df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
train_df['ImageId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()

print(train_df.shape)
train_df.head()

In [ ]:
mask_count_df = train_df.groupby('ImageId').agg(np.sum).reset_index()
mask_count_df.sort_values('hasMask', ascending=False, inplace=True)
print(mask_count_df.shape)
mask_count_df.head()

In [ ]:
sub_df = pd.read_csv('../input/severstal-steel-defect-detection/sample_submission.csv')
sub_df['ImageId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['ImageId'].unique(), columns=['ImageId'])

# Utility Functions

Source: https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode

## Mask encoding and decoding

In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(mask_rle, shape=(256,1600)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
def build_masks(rles, input_shape):
    depth = len(rles)
    height, width = input_shape
    masks = np.zeros((height, width, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, (width, height))
    
    return masks

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i])
            for i in range(depth)]
    
    return rles

## Loss function

Source for `bce_dice_loss`: https://lars76.github.io/neural-networks/object-detection/losses-for-segmentation/

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

# Sample Test

In [ ]:
sample_filename = 'db4867ee8.jpg'
sample_image_df = train_df[train_df['ImageId'] == sample_filename]
sample_path = f"../input/severstal-steel-defect-detection/train_images/{sample_image_df['ImageId'].iloc[0]}"
sample_img = cv2.imread(sample_path)
sample_rles = sample_image_df['EncodedPixels'].values
sample_masks = build_masks(sample_rles, input_shape=(256, 1600))

fig, axs = plt.subplots(5, figsize=(12, 12))
axs[0].imshow(sample_img)
axs[0].axis('off')

for i in range(4):
    axs[i+1].imshow(sample_masks[:, :, i])
    axs[i+1].axis('off')

# Data Generator

In [ ]:

# # example of random rotation image augmentation
# from numpy import expand_dims
# from keras.preprocessing.image import load_img
# from keras.preprocessing.image import img_to_array
# from keras.preprocessing.image import ImageDataGenerator
# from matplotlib import pyplot
# # load the image

# sample_filename = 'db4867ee8.jpg'
# sample_image_df = train_df[train_df['ImageId'] == sample_filename]
# sample_path = f"../input/severstal-steel-defect-detection/train_images/{sample_image_df['ImageId'].iloc[0]}"

# img = load_img(sample_path)
# # convert to numpy array
# data = img_to_array(img)
# # expand dimension to one sample
# samples = expand_dims(data, 0)
# # create image data augmentation generator
# datagen = ImageDataGenerator(rotation_range=90)
# # prepare iterator
# it = datagen.flow(samples, batch_size=1)
# # generate samples and plot
# pyplot.figure(figsize=(40,40))
# for i in range(9):
# 	# define subplot
# 	pyplot.subplot(330 + 1 + i)
# 	# generate batch of images
# 	batch = it.next()
# 	# convert to unsigned integers for viewing
# 	image = batch[0].astype('uint8')
# 	# plot raw pixel data
# 	pyplot.imshow(image, aspect='auto')
# # show the figure
# pyplot.show()


In [ ]:

from tensorflow.python.keras.utils.data_utils import Sequence


class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path='../input/severstal-steel-defect-detection/train_images',
                 batch_size=32, dim=(256, 1600), n_channels=1,
                 n_classes=4, random_state=2019, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        
        X = self.__generate_X(list_IDs_batch)
        
        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            return X, y
        
        elif self.mode == 'predict':
            return X

        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, list_IDs_batch):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        
        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            img_path = f"{self.base_path}/{im_name}"
            img = self.__load_grayscale(img_path)
            
            # Store samples
            X[i,] = img

        return X
    
    def __generate_y(self, list_IDs_batch):
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            image_df = self.target_df[self.target_df['ImageId'] == im_name]
            
            rles = image_df['EncodedPixels'].values
            masks = build_masks(rles, input_shape=self.dim)
            
            y[i, ] = masks

        return y
    
    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)

        return img
    
    def __load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.

        return img

In [ ]:
train_idx = [ 12350, 11212, 93, 12267, 7507, 5949, 4201, 10624, 11075, 2432, 352, 11787, 6425, 8035, 1370, 8785, 8071, 1871, 8553, 9365, 5270, 3667, 8102, 4467, 174, 3527, 10350, 9687, 7113, 3854, 9279, 4222, 9107, 3797, 3293, 6648, 8684, 11505, 6564, 1598, 12541, 5792, 10355, 5534, 8238, 9926, 2968, 3839, 9741, 7021, 11319, 9392, 11788, 6435, 4216, 9260, 11088, 2243, 958, 7038, 10736, 10705, 12461, 5070, 9153, 9324, 3233, 11204, 4015, 10062, 6754, 400, 10513, 5177, 10633, 11590, 10915, 6548, 4670, 9421, 1996, 7614, 4408, 1885, 3646, 11253, 11096, 1607, 11482, 2222, 9408, 185, 703, 11544, 4288, 4337, 6520, 12133, 7284, 6924, 6279, 5457, 10862, 3364, 4495, 11881, 9615, 10593, 3033, 11963, 1001, 4305, 10965, 2008, 6064, 11581, 8202, 7315, 3541, 11, 3169, 1912, 1075, 9583, 5677, 9940, 7984, 8344, 3624, 9838, 6502, 5489, 1945, 3891, 9015, 2042, 11746, 7224, 5618, 10404, 8979, 6555, 6490, 8805, 735, 11840, 579, 9836, 11178, 8019, 605, 11596, 10139, 5503, 5112, 1262, 11294, 8840, 4842, 1521, 11557, 4639, 6337, 1466, 10017, 4869, 942, 8999, 5869, 3800, 10232, 12130, 3373, 6284, 9355, 5663, 2605, 12097, 6787, 9410, 3670, 9774, 7453, 10419, 8220, 9627, 8709, 7168, 347, 3028, 9192, 6397, 1286, 10906, 2821, 11489, 9461, 6381, 6182, 12389, 6152, 6676, 9697, 6841, 6950, 6513, 6806, 10688, 9116, 64, 11140, 6165, 3701, 1650, 1701, 7597, 8737, 4251, 8799, 5299, 5417, 2669, 7231, 3283, 7702, 329, 9769, 10198, 10220, 7072, 6534, 5337, 6738, 6885, 7725, 8683, 5734, 10585, 9771, 12484, 10734, 8133, 11848, 10280, 8656, 4809, 2279, 10679, 397, 10018, 12428, 11720, 7982, 2076, 2817, 9068, 5613, 6317, 2976, 11970, 2514, 4967, 2611, 9247, 2873, 4252, 6429, 3831, 4750, 5340, 4455, 7049, 3490, 37, 9996, 2057, 9932, 7387, 11374, 4309, 201, 9618, 6012, 9438, 9766, 1613, 4489, 3684, 2929, 7434, 12314, 7263, 12192, 4150, 8559, 6412, 7891, 7981, 1530, 2586, 4723, 393, 11270, 10008, 9674, 2180, 11171, 5055, 7911, 6401, 11271, 4376, 9207, 5765, 6160, 571, 785, 4918, 10377, 9000, 12351, 7700, 10142, 34, 10572, 6497, 8450, 12251, 12280, 44, 12086, 3085, 12382, 11490, 9472, 1101, 1984, 9204, 11770, 4944, 12062, 4939, 10356, 12360, 3223, 1914, 8490, 3266, 5224, 8971, 2558, 9634, 1637, 14, 418, 1426, 7800, 6565, 1108, 11501, 836, 10114, 612, 6789, 11028, 9517, 183, 6026, 1775, 217, 8875, 10632, 8860, 4966, 8648, 3921, 19, 940, 10964, 4551, 5912, 7914, 4210, 2038, 7789, 6673, 4888, 10771, 9707, 2230, 6215, 3469, 63, 11689, 11775, 3168, 5845, 5266, 524, 1068, 8077, 9821, 10998, 11305, 10468, 11179, 522, 945, 1359, 3451, 8563, 5552, 8169, 4486, 9482, 9210, 7119, 3325, 7673, 6975, 2535, 11145, 7151, 10116, 7785, 12293, 401, 8163, 7015, 7988, 2626, 11157, 11236, 2406, 6396, 822, 7179, 7227, 4862, 2780, 301, 720, 10948, 10562, 9175, 9716, 9579, 10611, 897, 4458, 165, 554, 5254, 11618, 306, 11034, 11245, 5285, 7350, 7089, 619, 4082, 11928, 82, 6016, 2826, 6995, 10075, 12155, 1868, 2910, 5221, 2900, 5951, 1248, 11958, 3779, 4840, 7578, 7814, 4552, 10761, 5336, 4221, 293, 9837, 1501, 5497, 7303, 9625, 10251, 4193, 5703, 296, 7880, 2931, 2737, 9377, 4412, 1295, 3506, 9321, 3386, 300, 2096, 7190, 9744, 2719, 6407, 3640, 4872, 11430, 10829, 5877, 11629, 10319, 525, 2754, 9861, 3754, 10339, 7749, 5738, 2546, 5209, 2812, 2287, 7656, 6728, 7118, 10461, 11975, 76, 1904, 11163, 9344, 50, 7275, 2925, 2007, 10717, 7516, 10012, 3404, 5117, 4548, 5683, 2501, 7845, 4777, 148, 5185, 4366, 3703, 1058, 3089, 6622, 1883, 7874, 10651, 5424, 4492, 6992, 169, 3188, 8813, 6257, 8162, 918, 8498, 2333, 1160, 5711, 1326, 10646, 1790, 10687, 7232, 9348, 740, 2758, 8967, 7354, 7047, 7100, 12142, 623, 4899, 8853, 9810, 3850, 2264, 4328, 2844, 6045, 9513, 11315, 8751, 2181, 8107, 3765, 6793, 6629, 278, 4354, 6296, 11703, 30, 10262, 4602, 5828, 11114, 2196, 4550, 5997, 765, 2256, 9887, 2799, 9681, 12557, 11054, 1332, 6326, 8503, 10659, 2928, 1013, 5861, 10255, 4873, 12150, 3321, 5260, 7802, 6463, 5578, 8262, 2391, 10536, 3792, 3510, 2732, 9917, 9504, 5146, 611, 6537, 7041, 9231, 11789, 9156, 555, 890, 6033, 5072, 1026, 10782, 10252, 11722, 4461, 8872, 8666, 3240, 8692, 3472, 3208, 9066, 2118, 10798, 7562, 12451, 717, 10382, 553, 12111, 9776, 11886, 3445, 7962, 9222, 533, 4884, 11231, 12224, 2290, 5942, 84, 2527, 5086, 5641, 699, 7369, 2058, 8506, 3966, 9978, 3838, 1045, 10745, 6596, 4466, 9826, 10487, 11657, 5848, 11196, 4090, 2208, 9657, 3858, 10230, 4718, 11658, 12388, 2603, 5904, 5841, 6768, 5948, 7106, 907, 8732, 413, 7321, 6336, 8211, 10757, 4485, 9402, 7433, 11237, 4384, 7694, 9413, 4704, 8109, 12408, 10845, 9349, 9770, 11739, 1575, 9734, 9830, 12514, 9122, 6356, 6492, 9495, 5749, 3278, 8409, 6786, 7788, 4450, 4522, 5930, 6, 11299, 6363, 7290, 7146, 8156, 2193, 8866, 6710, 12349, 11621, 4624, 5601, 9466, 2097, 2203, 7769, 1952, 4547, 3302, 10148, 11427, 8746, 9963, 9187, 8856, 5482, 2794, 8956, 805, 4853, 10167, 9822, 170, 3884, 937, 7230, 11687, 9111, 873, 440, 5581, 807, 12175, 7806, 9686, 6739, 1947, 9862, 11697, 2677, 4526, 5421, 7921, 12262, 4481, 9950, 772, 4401, 4545, 1059, 2043, 11931, 12220, 264, 11936, 3737, 4801, 3880, 2427, 5919, 2070, 4463, 10070, 6325, 7753, 12423, 11918, 4452, 7996, 1042, 8977, 11573, 6507, 2282, 9714, 8375, 98, 3935, 12185, 9782, 1900, 7790, 10157, 4965, 9984, 195, 9400, 7824, 8609, 15, 7954, 8888, 6864, 1734, 5135, 684, 4901, 249, 1823, 9018, 4586, 6005, 477, 4508, 7926, 7112, 11539, 6467, 77, 5907, 4512, 244, 8937, 6832, 1813, 6731, 154, 12235, 8281, 1621, 8362, 12540, 2576, 10555, 8947, 7669, 5728, 1173, 3251, 339, 46, 8015, 3409, 4053, 8984, 8390, 10471, 10176, 3315, 4259, 1962, 2213, 4294, 7723, 7465, 490, 4751, 4754, 11546, 11705, 7848, 12449, 10384, 7061, 343, 8334, 6518, 5656, 9671, 1583, 6170, 7171, 1786, 10174, 4457, 5576, 7471, 7882, 1398, 3452, 5327, 2145, 8368, 8279, 3447, 7747, 4834, 9966, 1593, 9652, 7583, 2218, 7602, 2843, 9142, 3713, 2295, 3212, 8774, 10727, 10510, 8457, 12115, 2253, 1895, 10173, 10031, 6312, 258, 9946, 8500, 12338, 3985, 3523, 10996, 8784, 11769, 9751, 8804, 767, 8619, 3230, 5544, 12524, 7218, 1842, 5106, 4823, 582, 11945, 3763, 7735, 5083, 9809, 3637, 6483, 7324, 4636, 8939, 7881, 10805, 11382, 2472, 218, 5234, 10716, 4231, 222, 1955, 2130, 10568, 2399, 8910, 6344, 6255, 6506, 628, 5008, 10618, 11874, 323, 4106, 8903, 7328, 6499, 6819, 10997, 9072, 1078, 9711, 4505, 876, 6495, 8950, 4379, 3926, 9580, 9923, 6758, 7457, 6653, 787, 4945, 700, 7820, 3016, 6389, 11117, 1586, 386, 11394, 8355, 4286, 3707, 11290, 9450, 1839, 5294, 10574, 3544, 6308, 9896, 4311, 9527, 4520, 7677, 8592, 1343, 395, 7907, 1608, 7024, 6996, 9118, 6637, 7296, 5104, 1429, 6075, 10939, 7401, 8688, 11985, 4991, 6003, 1418, 8531, 10503, 9055, 2485, 9648, 9459, 10362, 6022, 2482, 11262, 2926, 3974, 6769, 480, 4027, 9145, 6481, 9505, 2171, 1803, 11955, 6980, 1364, 2864, 9685, 292, 7765, 998, 310, 8131, 3895, 889, 2471, 2654, 5441, 3973, 8069, 8568, 9568, 2739, 11959, 5876, 11062, 5330, 9255, 11045, 1257, 12414, 1580, 11626, 7688, 6509, 6472, 1852, 4080, 10163, 11810, 10835, 698, 8147, 2388, 7382, 1844, 4425, 4451, 11055, 1331, 11313, 10038, 5095, 11603, 9079, 10943, 5186, 425, 3630, 10202, 6219, 2922, 2298, 3830, 2733, 8736, 7092, 3726, 8421, 6822, 5446, 11189, 9030, 1164, 428, 2212, 9899, 2512, 1457, 12063, 2939, 6187, 6229, 11309, 2930, 399, 3255, 2320, 2565, 11184, 11999, 7577, 3049, 3502, 6698, 5345, 2397, 9534, 11847, 4441, 11815, 9229, 10932, 6963, 5917, 5460, 11887, 4555, 6517, 9216, 12222, 709, 6411, 12098, 5128, 3725, 6311, 11443, 933, 6755, 10867, 2589, 4030, 12278, 10972, 2840, 2886, 10667, 7955, 9971, 10615, 6370, 9570, 7329, 8383, 2554, 679, 3552, 615, 648, 3456, 6540, 5621, 11640, 7540, 7436, 3642, 12354, 11286, 1699, 368, 7520, 1999, 12065, 12379, 4942, 2729, 5756, 3945, 1935, 4432, 1270, 8783, 3129, 2834, 6569, 11333, 12558, 8489, 12099, 3138, 5619, 2979, 11458, 10032, 10756, 5821, 5076, 4672, 10866, 4731, 4033, 2871, 9679, 2098, 9431, 445, 6817, 2579, 10575, 6945, 8776, 10875, 11903, 10840, 5891, 10073, 6970, 353, 4327, 5736, 8600, 7174, 3711, 4344, 12520, 5608, 4765, 7858, 745, 11674, 7728, 4752, 1625, 3535, 6422, 5197, 2770, 5085, 2635, 2310, 6048, 9101, 5753, 3505, 7653, 4502, 3886, 4891, 8817, 239, 2355, 10366, 11306, 9214, 9261, 1610, 6300, 5745, 5449, 7743, 2041, 410, 10650, 9254, 1649, 8601, 3613, 8972, 5374, 7103, 3059, 5231, 10589, 9310, 11587, 10879, 5668, 5318, 101, 8491, 38, 11993, 9593, 6138, 2385, 11251, 10541, 3852, 1534, 10171, 4662, 9316, 5966, 2848, 5375, 7860, 287, 3094, 2890, 9832, 4136, 7373, 9173, 10787, 495, 6477, 7604, 8423, 7532, 7357, 4581, 8582, 11383, 4697, 10224, 9256, 7229, 10387, 2304, 9240, 11869, 4810, 6216, 11708, 9987, 10565, 5572, 12415, 7066, 10655, 5731, 9738, 2835, 10664, 10528, 2027, 8092, 6487, 2860, 4957, 10047, 11435, 6978, 3534, 3748, 227, 471, 4226, 4439, 5955, 7236, 4310, 9696, 6340, 9858, 10849, 4618, 1228, 4418, 9250, 11373, 3345, 6114, 9415, 6208, 2135, 5829, 4999, 5570, 7692, 3221, 7011, 1123, 6140, 10284, 2967, 3249, 11126, 7458, 2779, 4417, 4208, 2334, 8815, 11364, 1502, 4617, 1648, 8855, 1171, 12137, 6581, 4277, 2880, 7674, 8571, 9465, 9701, 1233, 4578, 8629, 7067, 12521, 6448, 11582, 6008, 10563, 11862, 9440, 11463, 6169, 8660, 6035, 3855, 8348, 6146, 6115, 1325, 10185, 1024, 10912, 8471, 11143, 245, 4246, 2570, 4992, 6432, 5577, 5717, 29, 3958, 11737, 6413, 11069, 2101, 9601, 7687, 4961, 4549, 3636, 11713, 6194, 11799, 1801, 6386, 1527, 5005, 3237, 2410, 8700, 7535, 1805, 416, 8537, 1335, 1044, 9063, 2148, 11432, 2480, 1260, 1723, 3538, 8387, 10477, 3047, 8453, 7213, 10887, 3043, 4063, 9367, 8558, 10743, 7514, 10827, 4124, 2664, 2941, 6702, 9395, 5148, 9071, 7162, 3976, 81, 7157, 11712, 3783, 1802, 9857, 294, 10184, 6672, 10307, 6922, 12286, 5271, 728, 10432, 1555, 10158, 3015, 4997, 10025, 11965, 6802, 6652, 8998, 8653, 653, 2365, 378, 8587, 4322, 9886, 8780, 5412, 3057, 3780, 5872, 4682, 366, 3072, 8185, 2012, 6827, 2918, 5898, 10437, 2959, 9855, 8083, 8464, 3521, 3658, 1235, 10178, 11330, 8710, 11420, 3370, 6912, 8325, 2405, 316, 10306, 6420, 349, 5088, 3574, 6528, 929, 11093, 11116, 7627, 9143, 4268, 9137, 5423, 8964, 283, 6708, 11317, 10774, 12440, 9494, 10431, 7841, 3071, 3744, 4592, 4448, 10460, 6349, 7247, 10806, 12309, 10802, 2806, 5409, 517, 7734, 1889, 2052, 10613, 11453, 10267, 9511, 11343, 12078, 7695, 8770, 712, 9129, 10649, 6052, 9319, 801, 5664, 3355, 1007, 1099, 8779, 11670, 11833, 1651, 5027, 10141, 11906, 9082, 2108, 1703, 10935, 1371, 8846, 11569, 9242, 10118, 2526, 948, 5804, 1785, 9596, 8046, 1960, 224, 4988, 6669, 11662, 5119, 7961, 6082, 5151, 1146, 11519, 10537, 7733, 8151, 6174, 1200, 1085, 3778, 5410, 10067, 1005, 4482, 2687, 8377, 7398, 7574, 4646, 9708, 2489, 8329, 9906, 3898, 12209, 7488, 10456, 12182, 2112, 6543, 7645, 4933, 11530, 9746, 10250, 10518, 7455, 6043, 7522, 2315, 75, 4893, 124, 1351, 8452, 3594, 1267, 3046, 10791, 11492, 11841, 2816, 5506, 7710, 125, 5091, 6358, 10129, 10775, 7403, 10730, 9050, 3117, 11793, 7260, 10376, 9641, 9001, 5098, 5036, 4174, 12081, 8745, 7090, 1254, 441, 6204, 5937, 867, 10519, 12259, 2030, 2691, 4092, 12536, 5645, 7470, 5080, 8720, 7405, 9203, 7429, 11506, 1476, 6318, 10414, 10140, 23, 6855, 10748, 5120, 11150, 9894, 12271, 7123, 7105, 12281, 7045, 1804, 8581, 11159, 4928, 8857, 9675, 8974, 10386, 12012, 4261, 2738, 1095, 4745, 6956, 6599, 4511, 7235, 9112, 2628, 12498, 3122, 2393, 1684, 6089, 2073, 6558, 4909, 3698, 4816, 5100, 6939, 4283, 5760, 3931, 6346, 9447, 2938, 10570, 10815, 934, 9902, 11536, 8180, 4453, 6444, 9234, 7447, 12096, 4766, 8504, 6951, 1623, 5445, 7298, 1915, 3243, 11227, 5009, 8099, 4867, 11433, 10271, 9990, 2744, 5210, 5507, 8435, 12416, 4291, 8021, 7893, 12197, 576, 10317, 6090, 10352, 11806, 591, 2828, 9964, 4811, 11425, 10911, 9829, 11643, 10678, 12123, 7756, 3739, 3987, 1685, 6069, 246, 1733, 10992, 1729, 9105, 4021, 6062, 5890, 4411, 10109, 1854, 6261, 11627, 10559, 2557, 7314, 2776, 4855, 3997, 4199, 5678, 9643, 1097, 4465, 12005, 10124, 3206, 4566, 1924, 1144, 5309, 8072, 8861, 2896, 4440, 11017, 12011, 1181, 11035, 697, 3929, 9880, 1491, 5956, 2356, 7258, 556, 2804, 7739, 9854, 7959, 7117, 1422, 12061, 1937, 10125, 10995, 9169, 2726, 11010, 6915, 5473, 3681, 4565, 6721, 8641, 11591, 11896, 9373, 7313, 7811, 1796, 8834, 12474, 6512, 12566, 6717, 9439, 10824, 4880, 8807, 7008, 6482, 8670, 4153, 9599, 233, 10467, 476, 12075, 3710, 4240, 10979, 7036, 5463, 6074, 666, 3124, 11911, 3652, 8593, 1898, 1826, 2543, 192, 7184, 6039, 6576, 1281, 11542, 658, 3449, 4521, 7265, 12529, 2014, 10223, 5771, 9110, 8277, 12462, 8724, 5582, 3885, 994, 4438, 6736, 10370, 2567, 3690, 5378, 3151, 11700, 8821, 9320, 7505, 5543, 3097, 12161, 11512, 7958, 923, 1846, 8952, 11876, 7966, 1463, 5123, 7510, 3405, 11078, 9725, 11819, 10494, 9007, 2609, 6314, 6324, 2313, 10289, 1595, 6164, 7712, 8699, 8400, 750, 10042, 2189, 10147, 8697, 10990, 9914, 408, 1532, 302, 6384, 4265, 6799, 11735, 4637, 9784, 8941, 10000, 11213, 1808, 1531, 3483, 8274, 1002, 804, 8293, 11579, 10369, 6410, 12437, 3806, 552, 5812, 9479, 7257, 5023, 11469, 9808, 2429, 7386, 8430, 5881, 9335, 7607, 7056, 4313, 3327, 8768, 5832, 5007, 7389, 6427, 6891, 7572, 3174, 6921, 8800, 5856, 3101, 10316, 3901, 6706, 4653, 649, 971, 6858, 2510, 5333, 6431, 4191, 12430, 8951, 9226, 9389, 6387, 11756, 220, 12343, 3967, 3406, 3864, 5860, 2507, 9677, 4627, 12473, 8567, 7444, 6971, 5964, 7755, 9718, 6877, 9374, 3178, 3520, 926, 8006, 1515, 12232, 10922, 11380, 9121, 11042, 12549, 8820, 3246, 12361, 3209, 311, 3856, 493, 9473, 11323, 382, 5096, 5504, 4181, 10475, 10959, 4998, 9031, 472, 5279, 9138, 6809, 5509, 3440, 901, 8835, 8120, 4010, 8380, 6238, 2566, 9604, 10321, 3214, 8051, 11419, 2398, 175, 6734, 4717, 8117, 5987, 9237, 6658, 10871, 10154, 10101, 11609, 1862, 4531, 3833, 2237, 5087, 312, 483, 9883, 4061, 384, 7708, 8174, 5329, 6812, 6580, 5002, 1745, 12067, 946, 1831, 376, 8175, 10551, 4462, 3241, 2297, 6845, 8756, 2649, 8261, 9428, 868, 160, 7659, 8936, 3031, 9190, 9039, 7388, 7970, 7003, 9292, 9554, 3860, 3761, 9779, 3040, 11784, 655, 1094, 586, 1712, 8178, 8440, 10238, 5899, 12000, 3620, 2906, 6360, 5495, 7339, 9177, 5998, 11883, 8564, 5910, 8831, 3529, 12273, 11846, 162, 8522, 3029, 6516, 2153, 6226, 8030, 661, 3522, 10085, 5696, 2714, 8771, 9040, 1006, 5208, 12037, 9475, 568, 4076, 8322, 5514, 10550, 2270, 9934, 1427, 4732, 2102, 10597, 3573, 12178, 8157, 2419, 3705, 10956, 9227, 1322, 5486, 3940, 52, 2170, 9271, 4057, 11752, 1388, 2798, 10540, 126, 4725, 2525, 12303, 1776, 9262, 139, 8882, 4488, 11407, 9908, 4142, 5638, 3425, 6557, 5187, 6294, 7430, 9539, 6144, 3694, 6816, 10021, 3836, 10878, 1313, 2479, 9106, 8084, 1897, 4188, 5630, 4543, 9045, 6825, 9198, 11578, 557, 2453, 7034, 12400, 1773, 12453, 8404, 3981, 4177, 1690, 7868, 12083, 5986, 8595, 10969, 10194, 111, 11233, 2132, 5858, 3530, 6441, 8509, 12290, 8442, 1444, 3526, 6465, 3444, 6928, 3580, 1529, 5761, 9871, 5136, 8584, 5622, 11681, 5433, 11395, 3152, 10074, 4469, 12213, 7161, 11471, 7116, 10066, 1415, 11112, 1919, 7144, 9965, 3766, 9995, 2785, 2923, 6906, 3306, 7292, 5607, 463, 1636, 4274, 243, 7312, 3896, 8744, 7920, 4529, 6286, 967, 8739, 811, 10418, 8395, 6824, 4573, 4720, 12356, 11446, 4878, 5430, 3742, 11033, 2678, 12279, 3888, 5768, 8561, 8081, 7536, 2540, 10961, 1041, 6643, 5968, 9041, 9753, 12395, 9977, 4800, 1093, 833, 1145, 10344, 721, 429, 9660, 3383, 3335, 2261, 9834, 6651, 3673, 11347, 173, 7950, 7832, 11274, 11465, 4375, 9919, 3847, 4736, 6310, 635, 11934, 4218, 5567, 6190, 9436, 713, 3556, 481, 7875, 3356, 208, 9844, 11243, 2978, 11352, 12551, 11781, 9003, 10832, 6430, 9890, 5269, 2361, 8997, 3605, 7823, 10010, 11894, 5971, 2899, 10003, 11170, 7037, 7466, 2465, 2996, 11856, 3804, 6551, 12479, 10258, 5947, 6602, 1186, 6617, 10405, 2390, 11377, 6997, 7946, 8224, 7657, 11046, 12503, 10571, 6228, 1582, 1992, 11487, 107, 3443, 754, 8196, 8326, 7476, 8706, 10522, 12230, 8646, 1938, 7383, 12141, 1266, 9350, 1488, 6096, 6618, 9036, 7967, 4340, 7310, 142, 2259, 544, 6120, 6025, 2184, 3106, 9553, 527, 5265, 3245, 4733, 3181, 6764, 3034, 5282, 6959, 7855, 7285, 7867, 8787, 4236, 10606, 4156, 7068, 6036, 3136, 2462, 11634, 598, 5367, 6646, 10209, 9103, 6371, 12417, 9301, 1498, 10398, 8428, 3268, 1536, 12320, 5225, 4459, 4421, 5218, 7026, 4036, 11277, 5226, 7863, 11012, 6455, 8625, 4542, 1812, 5319, 5115, 4540, 1948, 2006, 12032, 3894, 4587, 10786, 1639, 1665, 8425, 2796, 3102, 11747, 9174, 4248, 1647, 11524, 7070, 9212, 8869, 5245, 5102, 1968, 5585, 1243, 1223, 2777, 12553, 3820, 4012, 3704, 10557, 12074, 5859, 7740, 3133, 9462, 2293, 8934, 2083, 1880, 6767, 9322, 4879, 2161, 9476, 5154, 1847, 3943, 8992, 7351, 10926, 2749, 7336, 1087, 6056, 6121, 5531, 2800, 10403, 2271, 2990, 11558, 5153, 7719, 8123, 3815, 11266, 5022, 8778, 7899, 8635, 12446, 11556, 2727, 190, 4209, 1791, 3299, 12384, 12040, 12214, 5560, 914, 10014, 4844, 4224, 9458, 1959, 6316, 9624, 4378, 1560, 11094, 8393, 1988, 10741, 4302, 9653, 3171, 4925, 9243, 7549, 3347, 11351, 241, 1953, 276, 12072, 4897, 1544, 12047, 9994, 4217, 6450, 6542, 3454, 3468, 1105, 380, 6071, 3916, 6869, 8295, 10535, 7055, 4571, 5705, 10035, 10620, 7428, 12109, 6944, 10804, 6882, 10566, 3392, 634, 5650, 6533, 7726, 1309, 9299, 8703, 4779, 826, 4484, 11311, 8630, 5798, 11738, 2455, 11172, 9536, 9732, 71, 10856, 3157, 9807, 10170, 8177, 5172, 10683, 5524, 11779, 7497, 4976, 9949, 5042, 10689, 11611, 7379, 10297, 4647, 4380, 7478, 5565, 9286, 1136, 2063, 8345, 6167, 3691, 12542, 3755, 1225, 6620, 5013, 6254, 6225, 7590, 5820, 450, 11749, 12043, 7872, 10718, 2876, 6471, 9916, 10750, 11771, 8966, 8012, 5744, 9048, 5984, 113, 4404, 3376, 5122, 198, 11358, 4849, 8790, 3350, 7542, 5545, 1943, 2384, 8203, 4247, 6998, 3220, 7818, 984, 2066, 2341, 4707, 3189, 7207, 10453, 1311, 8197, 6172, 10430, 7007, 7249, 949, 12172, 6949, 4182, 7944, 7268, 141, 8263, 4914, 3794, 2252, 1717, 11128, 9974, 228, 5459, 6034, 3531, 8316, 394, 8399, 1150, 10239, 7512, 7393, 10270, 3035, 10137, 4253, 3512, 7431, 1319, 87, 6926, 1251, 7217, 11631, 8195, 7718, 2824, 2663, 1772, 1126, 10098, 10586, 8530, 12002, 8064, 9952, 7727, 9678, 10434, 5982, 7537, 8309, 10082, 4202, 3167, 8865, 103, 3542, 6241, 11943, 1755, 7831, 10156, 6514, 1416, 6399, 8389, 842, 5351, 3633, 1780, 4454, 5138, 12468, 8209, 5304, 7592, 9637, 4158, 1783, 7212, 11318, 8923, 3960, 11113, 5970, 11454, 4426, 12003, 5020, 10304, 968, 657, 11050, 3971, 7094, 9590, 9083, 136, 8244, 6986, 3513, 1663, 9750, 12007, 9115, 7163, 4900, 4655, 303, 11805, 4270, 3736, 3420, 10479, 1336, 2254, 4235, 1454, 2093, 6536, 10106, 5324, 6347, 6860, 6273, 9729, 1765, 1851, 5762, 3312, 10007, 1191, 4278, 9569, 10207, 3331, 1252, 5347, 2353, 3254, 1151, 5772, 4402, 10182, 5470, 10971, 10780, 9728, 9633, 7206, 6072, 11765, 7063, 11447, 9251, 5976, 4013, 1950, 10265, 5595, 11824, 10793, 2639, 10343, 7086, 4364, 7643, 5660, 2187, 8620, 3570, 8374, 7866, 11904, 5864, 7385, 1413, 10056, 10639, 10671, 10891, 2342, 6139, 7216, 5450, 5707, 10402, 8824, 12481, 5366, 5164, 9288, 8087, 10581, 9091, 2138, 7322, 11606, 12532, 11773, 1152, 12508, 10483, 453, 6392, 2211, 9980, 10580, 1730, 4940, 5037, 2597, 2818, 8668, 7518, 2921, 2715, 5097, 5825, 35, 7857, 6810, 11250, 5838, 3782, 9329, 6320, 5686, 6626, 5385, 9278, 6366, 6608, 7064, 618, 4497, 12467, 11922, 1038, 7225, 877, 7165, 6275, 7469, 6189, 4706, 9775, 620, 5699, 8919, 6625, 257, 3596, 12004, 7411, 7306, 2555, 4078, 8145, 1645, 11058, 10509, 886, 7680, 11391, 2740, 812, 1974, 9114, 12537, 10788, 1440, 1125, 10885, 9630, 473, 4799, 802, 5758, 375, 3153, 10719, 3426, 317, 3907, 132, 5637, 6240, 470, 8793, 3058, 7767, 8005, 448, 8682, 10874, 7320, 6613, 2002, 12352, 6530, 687, 5130, 8716, 8034, 7416, 1640, 9016, 6197, 12538, 9867, 3553, 9954, 8391, 838, 2499, 1735, 6013, 10699, 1268, 177, 3696, 7662, 6268, 2562, 8602, 262, 7441, 3952, 839, 273, 3009, 11356, 7360, 9928, 3007, 285, 6309, 10505, 7611, 1277, 1224, 7281, 9703, 4423, 10440, 10383, 9565, 3408, 6982, 10059, 5217, 8343, 6503, 2919, 4947, 10458, 8096, 11580, 4601, 11361, 8792, 6173, 3149, 2420, 11285, 4740, 11418, 11388, 3731, 2606, 6269, 4001, 8350, 5632, 2736, 4791, 1000, 5126, 122, 871, 1893, 10799, 85, 4746, 1894, 8070, 892, 9550, 610, 1385, 9125, 5573, 1816, 12121, 5334, 2872, 236, 12168, 12276, 3807, 5215, 4938, 8729, 3899, 3024, 383, 3675, 9097, 1142, 9884, 3218, 6556, 7640, 8610, 8276, 672, 5784, 10222, 11259, 3487, 10052, 12438, 33, 3137, 12392, 1153, 5788, 4533, 3682, 8717, 5620, 7523, 4658, 9215, 3799, 3913, 8170, 2418, 5624, 5359, 8142, 7757, 5137, 4924, 4881, 2382, 12129, 1368, 2769, 916, 4365, 3048, 9456, 5114, 2768, 6038, 3770, 688, 10594, 4562, 2571, 7054, 5685, 7949, 1132, 1788, 4005, 11155, 4115, 4428, 10334, 11892, 5759, 2637, 10298, 516, 7759, 1835, 7138, 7409, 4285, 254, 1485, 2703, 3609, 11408, 12441, 12038, 69, 6103, 1405, 566, 11494, 5434, 8013, 7609, 7425, 6667, 5811, 6589, 70, 665, 5894, 1231, 1386, 5611, 10643, 8608, 9024, 10538, 7361, 5866, 2997, 1828, 9445, 7375, 9726, 7136, 5099, 1928, 3518, 3073, 6250, 8845, 1737, 6974, 1715, 3261, 1128, 1102, 9430, 6644, 2755, 1249, 2428, 5396, 5229, 11025, 10738, 12268, 11065, 7252, 11884, 7936, 4534, 5826, 2858, 11076, 8570, 2877, 10348, 7335, 1098, 8159, 1564, 5671, 7018, 8284, 4070, 424, 8890, 7721, 8237, 2402, 433, 7460, 2660, 4152, 9253, 3878, 6046, 621, 3194, 10701, 11104, 4020, 5653, 993, 8669, 392, 237, 1708, 4098, 3968, 9337, 1513, 4694, 4626, 1753, 3653, 11488, 7792, 7145, 10999, 10328, 9559, 5183, 775, 5751, 10016, 2711, 1015, 221, 8510, 3587, 9668, 3575, 8251, 4937, 585, 6014, 1876, 11431, 9856, 7979, 3660, 10310, 5647, 11314, 4785, 1456, 10278, 6730, 1932, 9661, 723, 7220, 12381, 4195, 305, 11098, 10117, 1781, 7042, 8172, 4776, 5510, 8628, 8659, 8482, 7194, 9128, 7901, 7511, 8876, 5941, 5045, 11002, 9172, 1265, 2494, 2498, 2302, 10061, 1046, 51, 9524, 917, 117, 12148, 11576, 9006, 9379, 11376, 4223, 3638, 2722, 662, 790, 794, 6840, 8392, 645, 12317, 11827, 6523, 4783, 9012, 6081, 213, 12202, 4604, 671, 10379, 2646, 9874, 1404, 2089, 232, 4406, 6126, 2629, 4907, 2508, 4306, 2412, 8204, 203, 12494, 8877, 10322, 9767, 5199, 12234, 7238, 12277, 7241, 10985, 12560, 2547, 2407, 4955, 7648, 8412, 7420, 550, 7931, 1740, 4687, 11969, 5604, 9669, 5612, 3388, 3496, 4699, 10192, 3735, 1056, 7289, 3842, 1782, 1901, 7795, 11515, 12059, 5081, 256, 7732, 4806, 11371, 8796, 6813, 7331, 7196, 8327, 2988, 6985, 3175, 6707, 10677, 10088, 4669, 10385, 10034, 11295, 974, 883, 5522, 7291, 12496, 9468, 4518, 149, 2973, 7587, 806, 6385, 9664, 1585, 6234, 3749, 9380, 1387, 8360, 9736, 9484, 10446, 6252, 460, 6281, 2359, 4084, 6104, 905, 8980, 6880, 11547, 3564, 7242, 11679, 7763, 11731, 6327, 11559, 4047, 474, 6461, 11051, 8385, 6803, 8541, 2829, 11348, 12404, 9282, 3092, 6400, 4212, 737, 3724, 8114, 6619, 2561, 6638, 91, 1766, 12403, 10346, 7698, 12393, 5575, 6910, 11527, 9986, 6143, 9053, 4464, 7637, 9798, 10333, 5469, 2850, 7633, 7817, 6460, 11733, 10838, 5, 3811, 2157, 5110, 2081, 8397, 9378, 9193, 11764, 674, 1356, 9487, 4480, 12200, 9848, 6615, 11666, 11642, 5364, 8126, 846, 10520, 1654, 4341, 2377, 11690, 1066, 11005, 11410, 607, 1986, 5827, 10530, 513, 11897, 4588, 2281, 1635, 430, 8488, 11219, 6713, 4743, 5342, 7601, 4117, 4935, 8689, 2447, 693, 6757, 9968, 11175, 6814, 11110, 2905, 8914, 6116, 3280, 11387, 6894, 3882, 4612, 1163, 5960, 5781, 5322, 7486, 3951, 2354, 9223, 11660, 12112, 167, 5499, 6525, 188, 2907, 12073, 11415, 8113, 5462, 1985, 10631, 4501, 8755, 4026, 2504, 5306, 4843, 3509, 758, 12237, 7714, 2972, 11675, 8982, 5092, 10894, 1273, 10499, 7177, 3164, 3170, 8249, 9258, 8651, 6093, 12312, 11718, 3202, 9694, 1395, 3697, 7846, 6873, 8961, 504, 6785, 5589, 11757, 11063, 692, 11133, 1762, 8981, 5502, 2809, 1308, 5049, 12104, 4267, 2488, 6161, 6195, 6703, 2257, 342, 10886, 5852, 9688, 10796, 2914, 8332, 8229, 11954, 4846, 5833, 987, 5508, 6664, 9740, 102, 10661, 1727, 1972, 7547, 3229, 259, 894, 3365, 11234, 12227, 6927, 1079, 11755, 7435, 5633, 8963, 837, 5484, 3225, 2017, 4610, 9252, 7347, 9573, 9035, 6633, 9124, 7892, 6547, 2630, 4196, 3870, 11532, 6801, 3213, 3990, 6151, 1767, 7744, 1678, 6488, 8492, 8415, 8369, 2035, 7010, 2753, 10218, 10607, 11338, 8275, 9878, 3776, 11601, 10078, 1018, 5316, 1535, 1869, 135, 2250, 9388, 2389, 7717, 12024, 8356, 4357, 12176, 11758, 4171, 7002, 11872, 3100, 850, 12564, 531, 12324, 3298, 7553, 3273, 670, 8548, 7856, 7083, 4043, 6668, 1806, 3289, 2192, 2386, 2731, 12069, 8569, 8758, 11217, 7192, 12169, 6049, 10450, 1495, 4190, 1435, 1698, 6244, 8485, 8864, 11743, 5440, 1741, 5383, 1373, 12199, 5688, 423, 11508, 5606, 11750, 4185, 8517, 4398, 11766, 8859, 1853, 9435, 1381, 6606, 11977, 11529, 6102, 9150, 7941, 10855, 11613, 3511, 10166, 12196, 4634, 10546, 12387, 6011, 10493, 10974, 1970, 2276, 564, 4, 1131, 8181, 1369, 2713, 3176, 2466, 1525, 11535, 344, 7394, 11901, 1793, 1465, 12365, 8089, 5456, 5529, 134, 2676, 11724, 10472, 9973, 1039, 6600, 2668, 8549, 7879, 8221, 2018, 10303, 1837, 5241, 4620, 815, 10917, 4616, 5818, 3571, 7239, 464, 7898, 4798, 4814, 10978, 12162, 6749, 10115, 10842, 11653, 10186, 12294, 4161, 12459, 11203, 12510, 6147, 9690, 10940, 8207, 7082, 4477, 3832, 1820, 3400, 9032, 3366, 6059, 10024, 5961, 5785, 2029, 9131, 1995, 8439, 9609, 11238, 130, 11817, 3859, 9077, 7158, 7077, 982, 11331, 4456, 11130, 8340, 6283, 2742, 9582, 12247, 6747, 6552, 4003, 856, 4065, 3453, 2634, 11851, 2362, 2268, 5274, 9801, 11059, 10807, 9518, 5975, 10576, 12495, 8897, 6524, 9165, 1446, 6591, 3634, 1414, 3812, 10913, 10960, 1556, 3844, 164, 1400, 1392, 11976, 930, 4399, 4048, 7527, 3148, 10759, 12445, 7670, 10127, 5485, 4079, 11754, 7616, 10858, 7057, 3915, 5643, 911, 10769, 3829, 11160, 10417, 4861, 4419, 2942, 965, 12455, 10729, 11353, 4257, 8958, 8873, 2292, 9384, 2593, 2868, 3853, 5844, 9019, 12337, 8323, 391, 7626, 1632, 7104, 9092, 9645, 12033, 10795, 10276, 6403, 8111, 10753, 8812, 8693, 12219, 3803, 11221, 9640, 12434, 11188, 1561, 8333, 12531, 2783, 2442, 3296, 9587, 9211, 11060, 2443, 4263, 10921, 8054, 6445, 2855, 5308, 2025, 7060, 67, 4498, 10424, 9277, 5370, 2553, 8540, 9542, 2517, 572, 12485, 8406, 2920, 10257, 404, 6073, 6612, 6478, 8010, 8791, 12299, 297, 10019, 1484, 1592, 2643, 781, 6935, 1148, 3385, 9441, 2813, 10843, 9818, 3324, 9132, 3082, 7999, 898, 4407, 1471, 6009, 9850, 5593, 6362, 5579, 6777, 4165, 4794, 3872, 3287, 3359, 114, 8826, 5684, 7173, 1241, 4205, 11417, 11100, 2216, 5658, 8105, 11038, 4037, 1421, 9970, 1538, 191, 3577, 10203, 2165, 3498, 6966, 11003, 12245, 6994, 8020, 5953, 9154, 1294, 9781, 10360, 5554, 5865, 8854, 10684, 11989, 2692, 12106, 11987, 3879, 4851, 5574, 9799, 1397, 3450, 9022, 2142, 12184, 2490, 3377, 3720, 5394, 11795, 11099, 3599, 5281, 608, 6623, 7182, 11554, 5326, 59, 8100, 6868, 10656, 5790, 3549, 11275, 813, 2037, 5156, 9520, 1165, 3576, 9273, 8874, 9863, 1920, 9564, 5517, 726, 6185, 4148, 12335, 11507, 4709, 5408, 10698, 4436, 12302, 12475, 1193, 12341, 2851, 4126, 9622, 2662, 10103, 9835, 73, 9067, 10945, 1060, 3398, 1209, 7850, 8906, 2126, 6369, 7384, 8468, 1980, 8480, 11572, 9280, 4118, 3319, 5680, 1994, 2984, 5158, 9467, 10825, 7326, 7033, 3139, 274, 768, 9343, 7274, 11560, 3478, 10261, 7666, 3379, 10784, 664, 6047, 3631, 8342, 4091, 11068, 7380, 10365, 5635, 4119, 7345, 4114, 5435, 6920, 1452, 5816, 3018, 8243, 5246, 3582, 11950, 2529, 1441, 9507, 10179, 5298, 2933, 4300, 8098, 9577, 6905, 1744, 105, 9094, 1190, 3837, 8029, 9942, 4757, 9843, 9562, 1642, 5453, 6037, 8515, 54, 8408, 600, 10175, 41, 7132, 11263, 1646, 4323, 6391, 5974, 6979, 1034, 6605, 5779, 6270, 2587, 9020, 2857, 1307, 3052, 7745, 1221, 2269, 8414, 861, 5786, 2778, 10968, 1752, 6554, 1617, 2349, 17, 1285, 8946, 10808, 3706, 8459, 9628, 904, 8346, 8968, 7751, 10587, 12465, 2200, 9561, 8271, 5747, 1316, 6459, 8521, 3198, 2064, 9302, 10215, 8657, 2415, 5060, 5019, 12060, 10242, 2092, 11914, 6988, 5644, 12535, 8543, 3528, 9396, 4813, 6527, 7166, 4058, 12304, 1533, 1942, 776, 3393, 11235, 3236, 5830, 7839, 11719, 2051, 10092, 3387, 9969, 156, 9549, 10508, 3623, 2004, 1704, 11087, 6134, 1940, 3606, 1047, 896, 9176, 10112, 6861, 11102, 10899, 12567, 1887, 11962, 8881, 11288, 5202, 5994, 9612, 4077, 3144, 10765, 5657, 4953, 639, 11562, 6878, 5368, 7074, 8789, 11986, 2036, 7557, 10168, 7786, 6836, 12, 9148, 718, 5033, 106, 6774, 7912, 11224, 10254, 1430, 6202, 9937, 10465, 5648, 8513, 5849, 1861, 1611, 7524, 9217, 10893, 7276, 12486, 6149, 12218, 6610, 8199, 8577, 10859, 5720, 9406, 4908, 242, 3946, 11077, 12295, 11831, 9598, 4657, 1394, 9538, 6788, 990, 6184, 3772, 5551, 7544, 1693, 4974, 8535, 11933, 8291, 7989, 205, 9383, 1425, 4962, 7, 10561, 570, 11854, 7600, 9028, 11165, 12367, 5746, 8154, 5750, 4727, 4679, 11639, 11890, 9555, 8353, 6961, 10623, 65, 12105, 1464, 2964, 7993, 1445, 887, 10236, 8314, 7963, 11018, 7208, 7606, 6568, 3173, 3551, 5892, 9870, 9057, 8879, 7028, 9488, 6217, 11496, 3286, 9100, 5615, 2710, 8330, 9551, 3993, 1599, 3211, 9786, 4233, 6695, 11174, 677, 9328, 11941, 3545, 3281, 10151, 5535, 5167, 4692, 2306, 4590, 7256, 4282, 872, 8618, 6095, 5295, 10903, 3307, 10388, 10927, 12433, 4708, 1238, 11071, 7972, 7501, 6329, 11001, 10081, 1031, 6098, 3661, 295, 5168, 5886, 12144, 4804, 8575, 10539, 12229, 10848, 8711, 12095, 1494, 6948, 2531, 8253, 2819, 6640, 1468, 3012, 6960, 8150, 12419, 2491, 1949, 2015, 10349, 2300, 1756, 2226, 10259, 9385, 10569, 753, 5192, 8976, 7791, 7995, 12019, 9219, 6137, 1523, 12402, 10724, 5730, 1289, 6374, 8818, 8690, 2104, 6087, 9840, 3550, 12125, 2963, 10300, 6977, 757, 9351, 1009, 3011, 5695, 10686, 4824, 11843, 6263, 2369, 8317, 11326, 520, 12257, 11302, 10272, 2763, 11000, 12014, 3475, 8679, 10264, 11855, 2325, 2317, 5057, 1964, 3484, 6201, 7908, 6701, 10883, 9189, 5420, 5787, 1512, 5173, 1378, 1574, 3026, 4553, 1362, 5640, 10676, 1864, 5464, 7454, 1558, 1720, 4906, 3961, 0, 11396, 6040, 1843, 1377, 1710, 2158, 5349, 9042, 10113, 6110, 6475, 12201, 3461, 5701, 3857, 9879, 3788, 1321, 4272, 809, 12502, 7798, 12046, 7079, 10672, 3204, 10253, 12501, 6222, 1177, 10579, 10846, 7910, 11436, 865, 8434, 3876, 3432, 11328, 7938, 2705, 12396, 7691, 2981, 7783, 6680, 6942, 8518, 10554, 469, 7813, 436, 4741, 3998, 11108, 11850, 86, 10816, 1546, 6415, 5990, 3621, 4841, 2956, 8712, 11711, 6687, 10050, 5533, 4430, 7267, 10213, 7647, 10183, 763, 8164, 9514, 5713, 5985, 5983, 3986, 7356, 2718, 8419, 3712, 8431, 2088, 8764, 4167, 5068, 10340, 2725, 9140, 8995, 5851, 8884, 10652, 2086, 1380, 11138, 1548, 7156, 8200, 11040, 11522, 678, 479, 547, 6991, 3460, 10069, 3070, 9159, 1012, 5311, 12028, 2594, 6453, 12233, 1572, 2301, 5263, 6930, 10481, 2152, 9232, 7251, 9868, 7459, 2852, 10801, 8661, 3419, 5034, 3688, 10476, 4659, 5850, 3603, 4367, 3680, 7822, 2283, 2909, 8270, 11423, 5471, 523, 9327, 10524, 5124, 7613, 7088, 435, 11853, 9266, 4860, 7956, 4915, 706, 7625, 4431, 1758, 10249, 4557, 4833, 8978, 7871, 4678, 9332, 9390, 1032, 5580, 5381, 7148, 2544, 5617, 7280, 1496, 4996, 1596, 11252, 5718, 1549, 10279, 12261, 8155, 12165, 6732, 2080, 9171, 545, 4413, 7327, 8009, 2470, 1210, 10670, 6588, 12092, 3719, 9356, 6282, 7782, 5291, 3244, 7722, 9791, 7801, 5945, 6751, 7649, 5296, 3768, 7746, 5733, 3988, 11413, 5710, 9502, 9449, 12018, 8214, 1709, 11730, 2228, 6500, 4826, 11246, 597, 3617, 3561, 5452, 10896, 9064, 4905, 1692, 4422, 11476, 4822, 10195, 4255, 3265, 1657, 4663, 5292, 2167, 6562, 2913, 1433, 2619, 60, 6522, 1989, 7234, 9291, 8674, 7378, 7939, 859, 8886, 4176, 3818, 12301, 1489, 459, 4157, 3441, 12505, 5021, 6289, 10582, 3537, 10973, 10444, 1122, 6099, 3292, 3158, 6301, 1602, 3309, 841, 8073, 10511, 1358, 10030, 5667, 10711, 3076, 3348, 8210, 8667, 9463, 884, 7278, 7253, 12328, 8193, 7142, 7050, 9636, 4792, 2842, 12420, 92, 8436, 9009, 1554, 1683, 10873, 9446, 3394, 1702, 10308, 11298, 549, 7437, 4415, 1748, 11122, 9407, 7992, 8080, 10629, 7233, 334, 9130, 1062, 11616, 1603, 3903, 6650, 10089, 8241, 9448, 9754, 1751, 8959, 12425, 9655, 9098, 11437, 341, 2847, 6607, 5442, 78, 7905, 9752, 11837, 9597, 10301, 6342, 2497, 9206, 2947, 8363, 5051, 8189, 1301, 3488, 1483, 3746, 3781, 5583, 1784, 5344, 3865, 11723, 11807, 5511, 3201, 10393, 4768, 6367, 8909, 518, 2286, 454, 9002, 534, 9364, 6811, 40, 3357, 4981, 10778, 729, 2296, 1705, 1965, 977, 8268, 4319, 10610, 5914, 4273, 2023, 5564, 5562, 10626, 4558, 5113, 4671, 12559, 3217, 7464, 11392, 11845, 2958, 9948, 1208, 5427, 280, 5268, 6609, 8124, 8624, 2789, 5403, 6239, 2515, 778, 7709, 7563, 9706, 2666, 535, 10742, 320, 8043, 8192, 8536, 8658, 3822, 2884, 3262, 22, 9136, 543, 11768, 9626, 4784, 1246, 8596, 12454, 3996, 7244, 4796, 2263, 1202, 1997, 11916, 1350, 2901, 9017, 6291, 3093, 10975, 1672, 9925, 719, 7245, 3304, 5040, 8458, 1719, 10039, 5216, 12296, 2379, 3096, 7305, 4293, 7085, 11942, 4819, 9869, 8398, 8539, 1166, 8497, 3162, 5729, 2207, 1082, 4577, 2970, 4479, 4468, 9737, 8850, 2411, 5180, 9759, 9023, 12458, 9692, 3892, 6850, 8938, 10821, 6936, 11156, 2350, 6795, 12266, 2166, 10335, 1227, 4006, 7869, 8899, 2445, 7150, 4603, 9387, 4017, 9070, 9434, 2409, 7679, 5681, 11636, 9368, 4877, 248, 10022, 1019, 3132, 5556, 5419, 11923, 11800, 1065, 6890, 10132, 196, 4349, 5967, 8733, 7480, 9076, 11322, 5802, 3091, 11694, 6699, 9909, 66, 1941, 1348, 9721, 8647, 791, 11957, 12212, 8286, 4141, 5847, 12044, 5988, 685, 11292, 2694, 4206, 8264, 10004, 9804, 2523, 2575, 7812, 3294, 3257, 11247, 7353, 155, 8996, 8665, 8186, 4528, 5011, 1269, 11064, 11659, 288, 11964, 2717, 414, 12409, 1971, 11698, 1049, 3428, 3369, 8604, 6689, 11939, 746, 108, 11485, 9372, 9989, 9621, 11220, 5116, 6967, 5447, 1833, 11778, 7462, 5895, 6068, 2439, 11342, 3700, 4059, 7269, 5742, 2792, 3019, 11902, 3339, 1664, 8849, 3119, 5665, 8486, 8583, 11070, 10233, 2870, 3054, 5823, 12507, 5888, 5174, 4162, 3477, 5724, 8265, 2079, 10063, 11878, 8152, 5814, 3753, 4154, 3867, 1769, 6666, 3848, 4254, 7397, 5101, 9831, 1979, 8680, 1890, 5778, 3037, 2149, 5212, 6258, 10435, 10288, 6923, 5439, 7030, 5884, 1540, 4652, 2461, 7051, 8726, 5155, 7585, 10665, 4346, 6639, 1450, 3308, 1077, 6214, 11401, 11589, 8844, 4556, 9309, 11282, 11814, 5553, 8487, 7023, 1739, 11468, 650, 10634, 6213, 6874, 4389, 6285, 8190, 1014, 1514, 2702, 6357, 4420, 9614, 1170, 362, 3346, 4499, 2177, 4447, 2883, 7219, 12323, 11504, 3676, 204, 9921, 434, 5386, 8767, 10096, 8902, 1499, 2309, 7372, 4230, 4773, 6142, 1946, 1461, 3972, 3439, 2982, 12157, 12250, 6856, 364, 6642, 3239, 8606, 1264, 4388, 9900, 9486, 2444, 1091, 7915, 2856, 4574, 1376, 2425, 10177, 7620, 4952, 7348, 4668, 1168, 11216, 9109, 10949, 3187, 4956, 11528, 921, 11899, 369, 3340, 346, 11540, 9600, 8519, 1746, 4774, 3874, 9865, 6060, 3813, 7304, 10710, 9825, 1067, 9529, 1848, 6818, 2277, 1064, 1757, 3353, 6032, 12241, 5682, 11181, 7005, 6476, 4885, 2106, 3917, 12539, 11123, 9134, 11944, 5871, 6955, 4096, 4138, 511, 7937, 3482, 797, 1127, 12256, 529, 348, 279, 3030, 12145, 3950, 594, 5069, 7678, 5649, 2892, 6903, 2054, 9509, 10029, 2178, 6881, 7943, 8324, 8579, 8240, 1455, 7367, 1859, 12051, 2084, 2303, 3232, 10918, 3585, 10164, 2095, 10375, 6489, 3751, 997, 1140, 10605, 2898, 9304, 2071, 4817, 2125, 10162, 8205, 8061, 565, 7595, 2859, 10614, 62, 9783, 7035, 4854, 7084, 11668, 764, 10512, 5379, 10722, 5980, 5934, 4755, 8023, 2788, 315, 10525, 4219, 2351, 12373, 10314, 1612, 10578, 5855, 7594, 2323, 11868, 2741, 9312, 2328, 11161, 11052, 4913, 7368, 10482, 97, 10358, 10792, 12516, 8795, 1573, 630, 2322, 1172, 6020, 2998, 11762, 9756, 9820, 9526, 8822, 11979, 6535, 3581, 12565, 573, 3476, 1296, 5679, 10097, 1624, 7259, 1787, 11409, 5052, 11200, 12126, 8130, 3762, 546, 4882, 869, 3970, 8544, 7363, 690, 1396, 9493, 3297, 12410, 5157, 314, 10549, 1686, 11308, 6259, 6457, 3114, 277, 5740, 11836, 4684, 2709, 9205, 7246, 5048, 7443, 7154, 3349, 1263, 8079, 1795, 6989, 5387, 74, 9086, 2319, 116, 3081, 2134, 5134, 8743, 11677, 5235, 5133, 10766, 8018, 4009, 8429, 4040, 7838, 5525, 4025, 2113, 9892, 4536, 6343, 708, 455, 11307, 9639, 7632, 12413, 9152, 10941, 1866, 6756, 7665, 10041, 11255, 6784, 4701, 3403, 777, 2495, 10617, 11842, 6621, 2538, 11521, 7997, 12523, 1162, 9427, 3657, 10211, 407, 2927, 9983, 6791, 1922, 5748, 3036, 6990, 6125, 12318, 1245, 1021, 6571, 1439, 4595, 4280, 8652, 5807, 11825, 12372, 10206, 7622, 8782, 9893, 5258, 10110, 12053, 6958, 878, 10502, 45, 12499, 6929, 2220, 1675, 8678, 8965, 8443, 4980, 10636, 1589, 9275, 7781, 6248, 11354, 6830, 4567, 2874, 8076, 11426, 11439, 3548, 4166, 6442, 7528, 8319, 7020, 11429, 10144, 5652, 5527, 11813, 5809, 7396, 12366, 9508, 7027, 9158, 3584, 6157, 499, 5307, 6657, 4729, 10439, 255, 5655, 961, 5194, 2604, 12331, 4331, 770, 7810, 3270, 9757, 3053, 7440, 9424, 6888, 6292, 4347, 12167, 8612, 4808, 5929, 9735, 11561, 10216, 1857, 8226, 10351, 8501, 8250, 7195, 6028, 3351, 3900, 340, 5169, 11296, 9233, 6937, 1526, 10953, 10364, 9151, 10080, 10438, 11780, 5315, 11566, 12333, 9087, 2784, 4178, 5732, 7663, 6438, 11340, 6236, 1341, 590, 7854, 10532, 1329, 11605, 3120, 11664, 3759, 10812, 11921, 8049, 9270, 6050, 10291, 7191, 4116, 1220, 8433, 10410, 11604, 5300, 5314, 11452, 8578, 11390, 1360, 3135, 6661, 9213, 10341, 1563, 5414, 10988, 624, 398, 6456, 3546, 12252, 12375, 6480, 4985, 7485, 11673, 9167, 4730, 3380, 11462, 3197, 9188, 2048, 489, 6206, 6027, 2204, 5477, 9644, 2129, 421, 6092, 10993, 10399, 9567, 7664, 950, 3014, 9139, 3390, 5528, 7903, 2468, 853, 11706, 10068, 2082, 12034, 10892, 12518, 5794, 5312, 9851, 9182, 4563, 1107, 8928, 10136, 2760, 11651, 4403, 2223, 6766, 10951, 5476, 1618, 11655, 5284, 7448, 5107, 5198, 629, 5389, 1967, 6953, 6727, 11740, 4262, 8292, 3871, 8843, 823, 12329, 2139, 11291, 11971, 7754, 7500, 1713, 5822, 6148, 7556, 3115, 7073, 7016, 1643, 11398, 4920, 7916, 1092, 10970, 9761, 1179, 11570, 964, 8232, 3918, 6462, 6437, 9027, 11168, 8165, 756, 10831, 7808, 2240, 10598, 5082, 8306, 11715, 8426, 12101, 11449, 9658, 8611, 5377, 2532, 6175, 11947, 6763, 200, 11988, 5475, 184, 8461, 12552, 1492, 10401, 9059, 11241, 10028, 9418, 4728, 8128, 8222, 5203, 11187, 2560, 4493, 12260, 12342, 8708, 9073, 5697, 2811, 8867, 4308, 1303, 9432, 9510, 3020, 4064, 1913, 7807, 11019, 4964, 304, 1721, 5063, 11946, 11244, 4946, 927, 12321, 6772, 4530, 10601, 9160, 11821, 6563, 9259, 5355, 10543, 2932, 4290, 7652, 9008, 1503, 2888, 10841, 8848, 1199, 3013, 11714, 10361, 11725, 9485, 8063, 372, 2631, 10668, 11467, 8993, 5950, 1328, 12041, 6823, 9147, 4050, 2265, 8328, 3242, 2260, 6166, 5518, 7538, 6304, 11047, 5722, 2620, 350, 10191, 2602, 2068, 6916, 3328, 309, 9945, 6833, 2231, 8134, 11925, 11649, 10421, 11599, 2452, 521, 1207, 8613, 1799, 11871, 7784, 6247, 4847, 3271, 2464, 1312, 451, 11440, 1841, 951, 5958, 2592, 12355, 2085, 8379, 11893, 11555, 10884, 1923, 6546, 4085, 7624, 8512, 4345, 9853, 2172, 705, 8223, 6051, 4894, 7928, 3320, 4187, 9264, 575, 9951, 4446, 1282, 2771, 11477, 707, 704, 3332, 9920, 1619, 4160, 1323, 5938, 9727, 12298, 7738, 1393, 6145, 10221, 10789, 2539, 5962, 12031, 4111, 10936, 5458, 5542, 6587, 1845, 7551, 10256, 12088, 331, 6298, 11761, 11716, 12411, 7069, 2244, 9523, 7000, 9117, 4228, 8361, 8496, 8741, 5989, 1726, 7059, 8484, 6577, 11759, 5416, 12371, 377, 3087, 11397, 7705, 3399, 4093, 2175, 10286, 4674, 6753, 8208, 6218, 11198, 3740, 786, 6984, 4095, 4923, 11909, 10326, 8896, 4645, 5176, 6454, 12506, 814, 8040, 848, 6414, 8280, 1363, 9722, 10285, 4100, 1858, 2493, 89, 11564, 9307, 1505, 3598, 4066, 5883, 888, 2881, 5196, 12519, 11585, 2413, 261, 5372, 7169, 3470, 8341, 7896, 3300, 10916, 5692, 10851, 8555, 1183, 2348, 1696, 8216, 7143, 8135, 625, 4392, 2249, 4393, 3224, 1028, 11628, 5105, 4680, 4028, 2251, 2641, 11920, 11776]
val_idx = [ 9617, 8545, 9099, 452, 10934, 9673, 1824, 1827, 2069, 6521, 8761, 5360, 4859, 3424, 2372, 2659, 4094, 9141, 3395, 8176, 6086, 6361, 1666, 8511, 12334, 3764, 12208, 3693, 8773, 1460, 8116, 11637, 10800, 7561, 7477, 12180, 6017, 7883, 8308, 2186, 7819, 12543, 5520, 7894, 9056, 5479, 9631, 411, 962, 4943, 10094, 5536, 1399, 7923, 3155, 4445, 7071, 7504, 11095, 9993, 4775, 8763, 2404, 12452, 11015, 3819, 7295, 9877, 5393, 11135, 8376, 8495, 2581, 3485, 5044, 6715, 1022, 7531, 4108, 5490, 6797, 4986, 10504, 12311, 9437, 5933, 11614, 4921, 12322, 4831, 10153, 3458, 4333, 8149, 6066, 8136, 8335, 3955, 6573, 5243, 11525, 3906, 8740, 8470, 1083, 5483, 12472, 10931, 3618, 10422, 11207, 11256, 3977, 11148, 5539, 7133, 11459, 8339, 4963, 2197, 6947, 7075, 847, 1888, 1480, 10861, 3446, 12483, 8777, 733, 12064, 7521, 9498, 10558, 4675, 8904, 8045, 1261, 4435, 2291, 4372, 1659, 8605, 10107, 9272, 7575, 3381, 2723, 7925, 8525, 7768, 8529, 1874, 3260, 1174, 5084, 10739, 5634, 3959, 922, 9812, 8371, 1203, 6862, 11898, 8639, 8483, 651, 4654, 8252, 3180, 9695, 6946, 5262, 12357, 9815, 1522, 10396, 5708, 7667, 5920, 8650, 11683, 4951, 10345, 7222, 322, 9305, 3471, 5398, 4391, 12527, 3627, 2730, 5301, 11652, 4381, 6796, 6908, 739, 1906, 3160, 2815, 1293, 4782, 5093, 7122, 12264, 3474, 7134, 5795, 3849, 10857, 8242, 3683, 10803]


In [ ]:
BATCH_SIZE = 8

# train_idx, val_idx = train_test_split(
#     mask_count_df.index, random_state=2019, test_size=0.15
# )

train_generator = DataGenerator(
    train_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4
)

val_generator = DataGenerator(
    val_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4
)
mask_count_df
len(list(train_idx) + list(val_idx))

# Model Architecture

The following model is directly taken from this awesome kernel: https://www.kaggle.com/jesperdramsch/intro-chest-xray-dicom-viz-u-nets-full-data

In [ ]:
def build_model(input_shape):
    inputs = tf.keras.layers.Input(input_shape)

    c1 = tf.keras.layers.Conv2D(8, (3, 3), activation='elu', padding='same') (inputs)
    c1 = tf.keras.layers.Conv2D(8, (3, 3), activation='elu', padding='same') (c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2)) (c1)

    c2 = tf.keras.layers.Conv2D(16, (3, 3), activation='elu', padding='same') (p1)
    c2 = tf.keras.layers.Conv2D(16, (3, 3), activation='elu', padding='same') (c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2)) (c2)

    c3 = tf.keras.layers.Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
    c3 = tf.keras.layers.Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2)) (c3)

    c4 = tf.keras.layers.Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
    c4 = tf.keras.layers.Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = tf.keras.layers.Conv2D(64, (3, 3), activation='elu', padding='same') (p4)
    c5 = tf.keras.layers.Conv2D(64, (3, 3), activation='elu', padding='same') (c5)
    p5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2)) (c5)

    c55 = tf.keras.layers.Conv2D(128, (3, 3), activation='elu', padding='same') (p5)
    c55 = tf.keras.layers.Conv2D(128, (3, 3), activation='elu', padding='same') (c55)

    u6 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c55)
    u6 = tf.keras.layers.concatenate([u6, c5])
    c6 = tf.keras.layers.Conv2D(64, (3, 3), activation='elu', padding='same') (u6)
    c6 = tf.keras.layers.Conv2D(64, (3, 3), activation='elu', padding='same') (c6)

    u71 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u71 = tf.keras.layers.concatenate([u71, c4])
    c71 = tf.keras.layers.Conv2D(32, (3, 3), activation='elu', padding='same') (u71)
    c61 = tf.keras.layers.Conv2D(32, (3, 3), activation='elu', padding='same') (c71)

    u7 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c61)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
    c7 = tf.keras.layers.Conv2D(32, (3, 3), activation='elu', padding='same') (c7)

    u8 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(16, (3, 3), activation='elu', padding='same') (u8)
    c8 = tf.keras.layers.Conv2D(16, (3, 3), activation='elu', padding='same') (c8)

    u9 = tf.keras.layers.Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    c9 = tf.keras.layers.Conv2D(8, (3, 3), activation='elu', padding='same') (u9)
    c9 = tf.keras.layers.Conv2D(8, (3, 3), activation='elu', padding='same') (c9)

    outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='sigmoid') (c9)

    model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
    
    model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_coef])
    
    return model

# Training

Unhide below to see full summary:

In [ ]:
model = build_model((256, 1600, 1))
model.summary()

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'model.h5', 
    monitor='val_dice_coef', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)


history = model.fit_generator(
    train_generator,
    validation_data=val_generator,
    callbacks=[checkpoint],
    use_multiprocessing=False,
    workers=1,
    epochs=70,
    class_weight=[5.741360089186176, 20.850202429149796, 1.0, 6.429463171036205]
)

In [ ]:
with open('history.json', 'w') as f:
    json.dumps(str(history.history))

history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['dice_coef', 'val_dice_coef']].plot()

In [ ]:
model.load_weights('model.h5')
test_df = []

for i in range(0, test_imgs.shape[0], 500):
    batch_idx = list(
        range(i, min(test_imgs.shape[0], i + 500))
    )
    
    test_generator = DataGenerator(
        batch_idx,
        df=test_imgs,
        shuffle=False,
        mode='predict',
        base_path='../input/severstal-steel-defect-detection/test_images',
        target_df=sub_df,
        batch_size=1,
        n_classes=4
    )
    
    batch_pred_masks = model.predict_generator(
        test_generator, 
        workers=1,
        verbose=1,
        use_multiprocessing=False
    )
    
    for j, b in tqdm(enumerate(batch_idx)):
        filename = test_imgs['ImageId'].iloc[b]
        image_df = sub_df[sub_df['ImageId'] == filename].copy()
        
        pred_masks = batch_pred_masks[j, ].round().astype(int)
        pred_rles = build_rles(pred_masks)
        
        image_df['EncodedPixels'] = pred_rles
        test_df.append(image_df)

In [ ]:
bla = pd.concat(test_df)
#c = bla.drop(columns='ImageId')
#c
#bla = None

In [ ]:
c = bla.drop(columns='ImageId')
c

In [ ]:
c.to_csv('submission.csv', index=False)